In [ ]:
#トレンドが発生しやすい時間帯の研究
!pip install yfinance
!pip install matplotlib

In [ ]:
#以下は1分変動率の絶対値を1時間ごとに足し合わせて、７日間の平均をとったもの。

#13時に底を打ち上昇、16時からさらに上昇し、２０時にさらに上昇　1時にピークを迎える　4時ごろまで下り、１０時からさらもう一段下る。

#13時、16時、21時から参加者が増え始めて、１時から参加者が減り始め、１０時ごろからもう一段参加者が減ることが推測される。

#トレンドの発生時間と比較すると興味深い。

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz


# Get historical Bitcoin data
data = yf.download("BTC-USD", interval="1m", period="7d")

# １ティック前のデータとの変化率
data['Price_Change'] = abs(data['Close'].pct_change())*100

# Extract hour from the index (Datetime) and add it as a new column
data['Hour'] = data.index.hour


# Convert GMT hours to JST hours by adding 9 hours
data['JST_Hour'] = (data['Hour'] + 9) % 24

# Calculate average price change by hour in JST
hourly_changes_jst = data.groupby('JST_Hour')['Price_Change'].mean()


# Plot the results
fig = plt.figure(figsize=(10, 6))  # グラフのサイズを10インチ×6インチに設定
plt.plot(hourly_changes_jst, linewidth=2)  # グラフの線幅を2に設定
plt.xlabel('Hour of the Day (JST)')
plt.ylabel('Average Price Change')
plt.title('Bitcoin Average Hourly Price Change (JST)')
plt.xticks(range(0, 24))  # x軸の目盛りを0から23までの整数値に設定
plt.xlim(0, 23)  # x軸の範囲を0から23までに設定
plt.grid(linewidth=0.5)  # グリッド線の幅を0.5に設定
plt.show()


# Convert the index to Japan timezone
data.index = data.index.tz_convert('Asia/Tokyo')

# Plot the data
fig = plt.figure(figsize=(28, 9))  # グラフのサイズを28インチ×9インチに設定
plt.plot(data.index, data['Close'])

# Set the x-axis labels to date and time format
ax = plt.gca()

# Set the DateFormatter with the timezone
jst = pytz.timezone('Asia/Tokyo')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M', tz=jst))

# Set x-axis major ticks every 3 hours
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))

plt.title("BTC-USD (Japan Time)")
plt.xlabel("Date and Time")
plt.ylabel("Price")
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Draw grid lines
ax.grid(which='major', linestyle='-', linewidth=0.5)

plt.show()
